In [ ]:
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
from skimage.transform import resize
from skimage.io import imread, imsave, imshow
from skimage import img_as_ubyte, img_as_float
from scipy.integrate import dblquad, quad

import time
import numpy as np
import matplotlib.pyplot as plt
import os
import gc

import CV_functions as cvf

file = '20210130l_0km.fits'
file_bias = 'bias20210130.fits'

st = time.perf_counter()
with fits.open(os.path.abspath(file_bias)) as fb, \
     fits.open(os.path.abspath(file)) as f:
    fb.info()
    f.info()
    frames_per_sec = 1 / f[0].header['FRATE']
    data = f[0].data.astype(np.float32) - np.mean(fb[0].data.astype(np.float32), axis=0)
    
print('time:', time.perf_counter()-st)

In [ ]:
# трешхолд по Отцу для бинаризации
def otsu_tresh(gray):
    pixel_number = gray.shape[0] * gray.shape[1]
    mean_weight = 1.0/pixel_number
    his, bins = np.histogram(gray, np.arange(0,257))
    final_thresh = -1
    final_value = -1
    intensity_arr = np.arange(256)
    np.seterr(invalid='ignore')
    for t in bins[1:-1]: 
        pcb = np.sum(his[:t])
        pcf = np.sum(his[t:])   
        Wb = pcb * mean_weight
        Wf = pcf * mean_weight
        mub = np.sum(intensity_arr[:t]*his[:t]) / np.float32(pcb)
        muf = np.sum(intensity_arr[t:]*his[t:]) / np.float32(pcf)
        value = Wb * Wf * (mub - muf) ** 2
        if value > final_value:
            final_thresh = t
            final_value = value
    return final_thresh

# средний кадр серии
def avr_data(data):
    return np.mean(data, axis=0)

# нормировка кадров серии
def norm_data(data):
    return (data)/(avr_data(data)) - 1
 
# бинаризация кадров
def im_bin(data):
    return (avr_data(data) > otsu_tresh(avr_data(data))) * int(255)

# вырезка зрачка
def otsu_res(data):
    return norm_data(data) * im_bin(data)

# обрезка зрачка в квадрат
def sq_cropp(data):
    mask = otsu_res(data)[0] != 0
    rows = np.flatnonzero((mask.any(axis=1))) 
    cols = np.flatnonzero((mask.any(axis=0)))
    squared = otsu_res(data)[:, rows.min():rows.max()+1, cols.min():cols.max()+1] 
    print('size of cropped image:', squared.shape)
    return squared 

def v(D, img, latency, frames_per_sec):
    k = D / img.shape[1]
    print('1 pixel equals to', k, 'meters')
    return (D / img.shape[1]) / (latency * frames_per_sec)

def cross_corr_fft(img1, img2):
#     corr = np.fft.fftshift(np.abs(np.fft.ifft2(np.fft.fft2(img1)*np.fft.fft2(img2).conjugate()))) # abs
    corr = np.fft.fftshift(np.real(np.fft.ifft2(np.fft.fft2(img1)*np.fft.fft2(img2).conjugate()))) # real
    corr /= np.max(corr)
    return corr

def cross_corr(img, latency):
    all_corr = np.zeros_like(img)
    for i in range(img.shape[0]-latency):
        all_corr[i] = cross_corr_fft(img[i], img[i+latency])
    avr_cross = np.mean(all_corr, axis=0)
    print('size of cross_corr image:', avr_cross.shape)
    return avr_cross
    
def draw_corr_with_velocity(img, D, latency, frames_per_sec):   
    x = np.round(v(D, img, latency, frames_per_sec)*np.linspace(-img.shape[0]//2, img.shape[0]//2, 5), 2)
    fig = plt.figure()
    ax = plt.axes()
    im = plt.imshow(img)
    ax.set_xticks(np.linspace(0, img.shape[1], 5))
    ax.set_yticks(np.linspace(0, img.shape[0], 5))
    ax.set_xticklabels(x, fontsize=12)
    ax.set_yticklabels(x, fontsize=12)
    ax.set_ylabel('Vy, m/s', fontsize=12)
    ax.set_xlabel('Vx, m/s', fontsize=12)
    cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
    plt.colorbar(im, cax=cax)
    ax.grid(color='grey', linestyle='--', linewidth=0.7, alpha=0.4)
#     plt.savefig(f'C:/astro/corr_test3.png', bbox_inches='tight')  

# доводка до квадрата k на k пикселей
def plain_square(image, k):
    img = image.copy()
    img = np.pad(img, ((k - data_test.shape[0], 0), (0, k - data_test.shape[1])), 'constant', constant_values=(0))
    return img

In [ ]:
st = time.perf_counter()
data = sq_cropp(data)
print('time:', time.perf_counter() - st)

plt.figure()
plt.imshow(data[0])

data_test = plain_square(data[0], 228)
print('size of square image:', data_test.shape)
del data
gc.collect()

In [ ]:
# догнать изображение до квадратного по большей стороне для всех данных (т.е. до 228 на 228)

# спросить может ли быть в DomeCam высота сопряжения отличной от 0 и -2 км

In [ ]:
'''работаем в [метрах]'''
D_m = 2.5 # диаметр телескопа, [м]
D_pix = data_test.shape[0] # диаметр телескопа в пикселях
nx = 1024 # кол-во пикселей 
f_scale = D_pix/(D_m*nx) # шаг по частоте, [м^-1]
z = 2000 # дистанция распространения, [м]
lambda_ = 500*pow(10, -9) # длина волны, [м]
delta = 0.01 # шаг апертуры (соотношение между 1 пикселем и метрами, т.е. 1 пикс = 0.01 м) 
const = 9.69*pow(10, -3)*16*pow(np.pi, 2) # константа перед интегралом 

'''
для примера выбрать Cn2 соответствующую качеству изображения, равным 1 угловой секунде на длине 500нм. 
Величину Cn2(z)dz рассматривать как одну величину, обозначим ее C

beta = 0.98 * lambda/r0 = 1 следовательно r0 = 0.98*lambda*206265
пусть мы наблюдаем в зените, тогда
r0 = (0.423*k^2 * C)^(-3/5), где k = 2*pi/lambda_
Отсюда выражаю С
'''
C = pow((0.98*lambda_)*206265, -5/3) / (0.423*pow(2*np.pi/lambda_, 2))
print('Cn2:', C)
print('f_scale:', f_scale)

xx, yy = np.meshgrid(np.linspace(-nx//2, nx//2-1, nx), np.linspace(-nx//2, nx//2-1, nx))
zz1 = np.sqrt((xx)**2 + (yy)**2) 
xx_scale = f_scale * xx 
yy_scale = f_scale * yy

def a1(fx, fy): #|f|^(-11/3)
    res = pow(np.sqrt(fx**2+fy**2), -11/3)
    return res

def a2(fx, fy, z, lambda_): # sin^2(pi*z*lambda*|f|^2)
    res = pow(np.sin(np.pi*z*lambda_*(fx**2+fy**2)), 2) / pow(lambda_, 2)
    return res

def a1_with_a2(fx, fy, z, lambda_):
    res = a1(fx, fy) * a2(fx, fy, z, lambda_)
    return res

def aperture_func(fx, fy, delta): # ,было А(f), исправил на A(-f), добавив минусы перед fx и fy
    res = np.abs(np.sinc(delta*(-fx))*np.sinc(delta*(-fy)))**2
    return res
    

def pod_integrant_main(fx, fy, z, lambda_, delta):
    res = a1_with_a2(fx, fy, z, lambda_) * aperture_func(fx, fy, delta)
    return res

# print(dblquad(pod_integrant_main, -np.inf, np.inf, -np.inf, np.inf, args=(xx_scale, yy_scale, z, lambda_, delta)))
res1 = C*const*pod_integrant_main(xx_scale, yy_scale, z, lambda_, delta)

img1 = a1_with_a2(xx_scale, yy_scale, z, lambda_)
img2 = pod_integrant_main(xx_scale, yy_scale, z, lambda_, delta)
img3 = aperture_func(xx_scale, yy_scale, delta)
X = img1[512, :]
X2 = img2[512, :]
X3 = img3[512, :]
Y = range(1024)

fig, ((ax_img, ax_img2), (ax_img3, ax_img4), (ax_img5, ax_img6)) = plt.subplots(3, 2, figsize=(25, 15))
ax_img.imshow(img1)
tmp = ax_img.imshow(img1)
fig.colorbar(tmp, ax = ax_img )
ax_img.set_title('a1*a2')
ax_img2.plot(X, Y)

ax_img3.imshow(img2)
tmp = ax_img3.imshow(img2)
fig.colorbar(tmp, ax = ax_img3)
ax_img3.set_title('a1*a2*A(-f)')
ax_img4.plot(X2, Y)

ax_img5.imshow(img3)
tmp = ax_img5.imshow(img3)
fig.colorbar(tmp, ax = ax_img5)
ax_img5.set_title('A(-f)')
ax_img6.plot(X3, Y)

In [ ]:
I0c = (data_test != 0) * int(1)
I0c = plain_square(I0c, nx)
print('I0c size:', I0c.shape)

corr = np.fft.fftshift(np.real(np.fft.ifft2(np.fft.fft2(I0c)*np.fft.fft2(I0c).conjugate())))

fig, (ax_img, ax_img2) = plt.subplots(1, 2, figsize=(25, 10))
ax_img.imshow(I0c, cmap='gray')
ax_img2.imshow(corr)
tmp = ax_img2.imshow(corr)
fig.colorbar(tmp, ax = ax_img2)

print('auto_corr size:', corr.shape)
print('max corr coords:', np.unravel_index(np.argmax(corr), corr.shape), ' and max corr value:', np.max(corr))
# центральное значение на автокорреляции = кол-во освещенных пикселей

In [ ]:
# подсчет gamma_0_0
gamma_0_0 = C * const * np.max(corr) * pod_integrant_main(xx_scale, yy_scale, z, lambda_, delta)
print('NaNs:', np.count_nonzero(np.isnan(gamma_0_0)))
print('gamma_0_0:', gamma_0_0[512][512])

In [ ]:
'''
data_test = data
print('data test shape:', data_test.shape)
st = time.perf_counter()

data_resized = resize(data_test, (data_test.shape[0], data_test.shape[1] // 3, data_test.shape[2] // 3), anti_aliasing=False)
data_avr_corr = cross_corr(data_resized, 4) 

kern = np.array([[ -1, -2, -1],
                 [ -2, 22, -2],
                 [ -1, -2, -1]])
data_box_filter = cvf.box_filter(data_avr_corr, kern)
data_resized_back = resize(data_box_filter, (data_test.shape[1], data_test.shape[2]), anti_aliasing=False)
# data_resized_back = resize(data_avr_corr, (data_test.shape[1], data_test.shape[2]), anti_aliasing=False)



print('time:', time.perf_counter() - st)

print('data resized back shape:', data_resized_back.shape)
draw_corr_with_velocity(data_resized_back, 2.5, 4, frames_per_sec)

data_uint = img_as_ubyte(data_resized_back)
print('otsu tresh:', otsu_tresh(data_uint))
data_otsu = (data_uint > otsu_tresh(data_uint)) * int(255)
fig = plt.figure()
ax = plt.axes()
im = plt.imshow(data_otsu)

# window = np.ones((7, 7))
# data_median = cvf.median_filter(data_otsu, window)
# fig = plt.figure()
# ax = plt.axes()
# im = plt.imshow(data_median)

del data_test
'''

In [ ]:
'''
st = time.perf_counter()
draw_corr_with_velocity(cross_corr(data, 4) , 2.5, 4, frames_per_sec)
print('time:', time.perf_counter() - st)
'''